사용자의 질문과 유사한 근거 문서를 여러 개 추출한 후, 중요한 순서대로 컨텍스트의 맨 앞쪽과 맨 뒤쪽에 배치하여 답변의 정확도를 향상

In [ ]:
# Chroma dimension 관련 에러 발생 시 실행
# from langchain_chroma import Chroma
# Chroma().delete_collection()

#### Long-Context Reorder 없이 유사 문서 출력

In [3]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma


# HuggingfaceEmbedding 함수로 Open source 임베딩 모델 로드
model_name = "snunlp/KR-SBERT-V40K-klueNLI-augSTS"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}
ko_embedding = HuggingFaceEmbeddings(
    model_name = model_name,
    model_kwargs = model_kwargs,
    encode_kwargs = encode_kwargs
)

texts = [
    "바스켓볼은 훌륭한 스포츠입니다.",
    "플라이 미 투 더 문은 제가 가장 좋아하는 노래 중 하나입니다.",
    "셀틱스는 제가 가장 좋아하는 팀입니다.",
    "이것은 보스턴 셀틱스에 관한 문서입니다."
    "저는 단순히 영화 보러 가는 것을 좋아합니다",
    "보스턴 셀틱스가 20점차로 이겼어요",
    "이것은 그냥 임의의 텍스트입니다.",
    "엘든 링은 지난 15 년 동안 최고의 게임 중 하나입니다.",
    "L. 코넷은 최고의 셀틱스 선수 중 한 명입니다.",
    "래리 버드는 상징적인 NBA 선수였습니다.",
]

# Chroma Retriever 선언
retriever = Chroma.from_texts(
    texts,
    embedding=ko_embedding
).as_retriever(
    search_kwargs={"k": 10}
)

# 유사도 기준으로 검색 결과 출력
query = "셀틱에 대해 설명해줘"
docs = retriever.invoke(query)
docs

/opt/anaconda3/envs/RAG_CODE/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Number of requested results 10 is greater than number of elements in index 9, updating n_results = 9


[Document(id='4f48ad06-aef8-43ba-86c3-d9208ace980e', metadata={}, page_content='셀틱스는 제가 가장 좋아하는 팀입니다.'),
 Document(id='ae652414-b234-4eaa-ae34-74356d241b5d', metadata={}, page_content='L. 코넷은 최고의 셀틱스 선수 중 한 명입니다.'),
 Document(id='3b3def7a-a61a-4783-a4aa-b1f7ada21fc1', metadata={}, page_content='이것은 보스턴 셀틱스에 관한 문서입니다.저는 단순히 영화 보러 가는 것을 좋아합니다'),
 Document(id='a6939e8c-76c0-4c2d-a633-4d84046b1b66', metadata={}, page_content='보스턴 셀틱스가 20점차로 이겼어요'),
 Document(id='1fb618d7-6f51-4729-82eb-07acaa1ad712', metadata={}, page_content='엘든 링은 지난 15 년 동안 최고의 게임 중 하나입니다.'),
 Document(id='9bc7960b-a24c-4c20-be33-3553a9cf27bf', metadata={}, page_content='플라이 미 투 더 문은 제가 가장 좋아하는 노래 중 하나입니다.'),
 Document(id='7086571e-e5f3-42d4-ad2b-8e441396ea5a', metadata={}, page_content='래리 버드는 상징적인 NBA 선수였습니다.'),
 Document(id='ab457140-256c-4b91-a659-5dd2fb7a2373', metadata={}, page_content='이것은 그냥 임의의 텍스트입니다.'),
 Document(id='e3160e96-f0ea-4b0d-b87a-a7eaca2172e1', metadata={}, page_content='바스켓볼은 훌륭한 스포츠입니다.')]

#### Long-Context Reorder를 활용한 유사 문서 출력

In [4]:
from langchain.document_transformers import LongContextReorder

# LongContextReorder 선언
reordering = LongContextReorder()

# 검색된 유사문서 중 관련도가 높은 문서를 맨앞과 맨뒤에 재정배치
reordered_docs = reordering.transform_documents(docs)
reordered_docs

[Document(id='4f48ad06-aef8-43ba-86c3-d9208ace980e', metadata={}, page_content='셀틱스는 제가 가장 좋아하는 팀입니다.'),
 Document(id='3b3def7a-a61a-4783-a4aa-b1f7ada21fc1', metadata={}, page_content='이것은 보스턴 셀틱스에 관한 문서입니다.저는 단순히 영화 보러 가는 것을 좋아합니다'),
 Document(id='1fb618d7-6f51-4729-82eb-07acaa1ad712', metadata={}, page_content='엘든 링은 지난 15 년 동안 최고의 게임 중 하나입니다.'),
 Document(id='7086571e-e5f3-42d4-ad2b-8e441396ea5a', metadata={}, page_content='래리 버드는 상징적인 NBA 선수였습니다.'),
 Document(id='e3160e96-f0ea-4b0d-b87a-a7eaca2172e1', metadata={}, page_content='바스켓볼은 훌륭한 스포츠입니다.'),
 Document(id='ab457140-256c-4b91-a659-5dd2fb7a2373', metadata={}, page_content='이것은 그냥 임의의 텍스트입니다.'),
 Document(id='9bc7960b-a24c-4c20-be33-3553a9cf27bf', metadata={}, page_content='플라이 미 투 더 문은 제가 가장 좋아하는 노래 중 하나입니다.'),
 Document(id='a6939e8c-76c0-4c2d-a633-4d84046b1b66', metadata={}, page_content='보스턴 셀틱스가 20점차로 이겼어요'),
 Document(id='ae652414-b234-4eaa-ae34-74356d241b5d', metadata={}, page_content='L. 코넷은 최고의 셀틱스 선수 중 한 명입니다.')]